In [15]:
import pandas as pd


from CLAPWrapper import CLAPWrapper
from esc50_dataset import ESC50
import torch.nn.functional as F
import numpy as np
from tqdm import tqdm
from sklearn.metrics import accuracy_score

import torch


In [16]:
df = pd.read_csv("../landscape/landscape_final.csv")

classes = list(set(df["label"]))

prompt = 'this is a sound of '
y = [prompt + x for x in classes]

class_count = len(classes)

In [17]:
class_dict = {k: v for v, k in enumerate(classes)}

In [37]:
# Load and initialize CLAP
weights_path = "../clap_weight/CLAP_weights_2022.pth"

# Setting use_cuda = True will load the model on a GPU using CUDA
clap_model = CLAPWrapper(weights_path, use_cuda=False)

# Computing text embeddings
text_embeddings = clap_model.get_text_embeddings(y)

# Computing audio embeddings
y_preds, y_labels = [], []


/kuacc/users/bbiner21/.conda/envs/clap/lib/python3.8/site-packages/torchlibrosa/stft.py:193: FutureWarning: Pass size=1024 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  fft_window = librosa.util.pad_center(fft_window, n_fft)
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expec

In [38]:

gt = []
pred = []

for i in tqdm(range(len(df.index))):
    x = "/datasets/audio-image/audios/audio_10s/" + df.iloc[i,1] + ".wav"
    
    cur_class = class_dict[df.iloc[i,0]]
    one_hot = torch.zeros((1,class_count))
    one_hot[0,cur_class] = 1.0 
    
    gt.append(cur_class)
    
    
#     x, _, one_hot_target = dataset.__getitem__(i)
    audio_embeddings = clap_model.get_audio_embeddings([x], resample=True)
    
    similarity = clap_model.compute_similarity(audio_embeddings, text_embeddings)
    y_pred = F.softmax(similarity.detach().cpu(), dim=1).numpy()
    
    pred.append(np.argmax(y_pred, axis=1)[0])
    y_preds.append(y_pred)
    y_labels.append(one_hot.detach().cpu().numpy())


100%|██████████| 1061/1061 [02:33<00:00,  6.92it/s]


In [23]:



# for i in tqdm(range(len(dataset))):
#     x, _, one_hot_target = dataset.__getitem__(i)
#     audio_embeddings = clap_model.get_audio_embeddings([x], resample=True)
#     similarity = clap_model.compute_similarity(audio_embeddings, text_embeddings)
#     y_pred = F.softmax(similarity.detach().cpu(), dim=1).numpy()
#     y_preds.append(y_pred)
#     y_labels.append(one_hot_target.detach().cpu().numpy())

y_labels, y_preds = np.concatenate(y_labels, axis=0), np.concatenate(y_preds, axis=0)
acc = accuracy_score(np.argmax(y_labels, axis=1), np.argmax(y_preds, axis=1))
print('ESC50 Accuracy {}'.format(acc))

"""
The output:

ESC50 Accuracy: 82.6%

"""


ESC50 Accuracy 0.4458058435438266


'\nThe output:\n\nESC50 Accuracy: 82.6%\n\n'

In [25]:
gt = []
x = "/datasets/audio-image/audios/audio_10s/" + df.iloc[0,1] + ".wav"

cur_class = class_dict[df.iloc[0,0]]
one_hot = torch.zeros((1,class_count))
one_hot[0,cur_class] = 1.0 

gt.append(cur_class)


#     x, _, one_hot_target = dataset.__getitem__(i)
audio_embeddings = clap_model.get_audio_embeddings([x], resample=True)

similarity = clap_model.compute_similarity(audio_embeddings, text_embeddings)

In [31]:
y_pred = F.softmax(similarity.detach().cpu(), dim=1).numpy()


In [35]:
np.argmax(y_pred, axis=1)[0]

3

In [41]:
from sklearn.metrics import confusion_matrix

matrix = confusion_matrix(gt, pred)
matrix.diagonal()/matrix.sum(axis=1)

array([0.28571429, 0.35164835, 0.7877095 , 0.59615385, 0.01639344,
       0.93243243, 0.93292683, 0.03092784, 0.4       ])

In [42]:
matrix

array([[  8,   2,   0,   0,   0,   0,  18,   0,   0],
       [  5,  64,   1,  11,   0,   0, 100,   1,   0],
       [  1,   1, 141,   5,   2,   3,  23,   1,   2],
       [  2,   1,   0,  31,   0,   1,  15,   0,   2],
       [ 70,  51,   0,   0,   3,   2,  40,  17,   0],
       [  1,   1,   0,   3,   0,  69,   0,   0,   0],
       [  2,   1,   7,   0,   0,   0, 153,   0,   1],
       [ 30,  85,   0,   1,   0,   0,  72,   6,   0],
       [  1,   0,   0,   1,   0,   1,   0,   0,   2]])

In [43]:
y

['this is a sound of waterfall burbling',
 'this is a sound of wind noise',
 'this is a sound of fire crackling',
 'this is a sound of thunder',
 'this is a sound of squishing water',
 'this is a sound of underwater bubbling',
 'this is a sound of raining',
 'this is a sound of splashing water',
 'this is a sound of explosion']

In [44]:
classes

['waterfall burbling',
 'wind noise',
 'fire crackling',
 'thunder',
 'squishing water',
 'underwater bubbling',
 'raining',
 'splashing water',
 'explosion']